In [184]:
import pandas as pd 
import dash
import dash_cytoscape as cyto
from dash import html, dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
from jupyter_dash.comms import _send_jupyter_config_comm_request

In [185]:
_send_jupyter_config_comm_request()

In [187]:
JupyterDash.infer_jupyter_proxy_config()

In [188]:
nodelist = [1,2,3,4,5,6]
edgelist = [(1,2),(1,3),(1,4),(1,5),(1,6),(2,3)]

graphitems = []

for i in nodelist:
    dashnode = {'data': {'id': str(i), 'label': str(i)}}
    graphitems.append(dashnode)

for i in edgelist:
    dashedge = {'data': {'source': str(i[0]), 'target': str(i[1])}}
    graphitems.append(dashedge)

In [189]:
app = JupyterDash(__name__)

app.layout = html.Div([
    cyto.Cytoscape(
        layout={'name': 'cose'},
        elements=graphitems
    )
])

In [190]:
app.run_server(debug=True)

Dash app running on https://jupyter.idre.ucla.edu/user/opheliadong@gmail.com/proxy/8050/


In [191]:
app = JupyterDash(__name__)

In [192]:
app.layout = html.Div([

    dcc.Dropdown(
        id='dropdown-update-layout',
        value='cose',
        options=[
            {'label': 'Grid', 'value': 'grid'},
            {'label': 'Random', 'value': 'random'},
            {'label': 'Circle', 'value': 'circle'},
            {'label': 'Cose', 'value': 'cose'},
            {'label': 'Concentric', 'value': 'concentric'}
        ]
    ),
    cyto.Cytoscape(
        id='cytoscape-compound',
        layout={'name': 'cose'},
        elements=graphitems
    )
])

@app.callback(Output('cytoscape-compound', 'layout'),
              Input('dropdown-update-layout', 'value'))
def update_layout(layout):
    return {'name': layout}

In [193]:
app.run_server(debug=True)

Dash app running on https://jupyter.idre.ucla.edu/user/opheliadong@gmail.com/proxy/8050/


In [194]:
app = JupyterDash(__name__)

In [195]:
app.layout = html.Div([
    dcc.Dropdown(
        id='dropdown-update-layout',
        value='cose',
        options=[
            {'label': 'Grid', 'value': 'grid'},
            {'label': 'Random', 'value': 'random'},
            {'label': 'Circle', 'value': 'circle'},
            {'label': 'Cose', 'value': 'cose'},
            {'label': 'Concentric', 'value': 'concentric'}
        ]
    ),
    cyto.Cytoscape(
        id='cytoscape-compound',
        layout={'name': 'cose'},
        elements=graphitems,
        
        stylesheet=[
            {
                'selector': 'node',
                'style': {
                    'background-color': 'green',
                    'content': 'data(label)',
                }
            },
        ]
    )
])

@app.callback(Output('cytoscape-compound', 'layout'),
              Input('dropdown-update-layout', 'value'))
def update_layout(layout):
    return {'name': layout}

In [196]:
app.run_server(debug=True)

Dash app running on https://jupyter.idre.ucla.edu/user/opheliadong@gmail.com/proxy/8050/


#### Below are the network visualization of the Romeo and Juliet play. 

#### Node: character names & the total number of times they appear
#### Edge: the least number of times that both characters appear in a scene
#### Example: for the string 'romeo romeo romeo juliet juiet,' Node'romeo' appear 3 times, Node 'juliet' appears 2 times, edge ('romeo, juliet') appears 2 times. 

In [197]:
with open('Shakespeare.txt') as f:
    text = f.read()

#### This text has a prologue, 5 acts, and a few words by the publisher after the last scene ends. 

#### First, I will take out the publisher's words at the end. 
#### Then, I will take the prologue out from the text since a quick inspection showed that the prologue does not include any character's name.

In [198]:
text_cleaned = text.split('THE END')[0]
text_cleaned = text_cleaned.split('ACT')[1:6]

In [199]:
character_list = ['Escalus','Paris','Montague','Capulet','An old Man','Romeo',
                  'Tybalt','Mercutio','Benvolio','Tybalt','Friar Laurence',
                  'Friar John','Balthasar','Abram','Sampson','Gregory','Peter','Lady Montague',
                  'Lady Capulet','Juliet','Nurse to Juliet']

In [200]:
# Each element in the list text_cleaned in an act
len(text_cleaned)

5

In [201]:
connections = {}
for i in range(len(character_list)-1):
    for j in range(i+1,len(character_list)):
        connections[(character_list[i],character_list[j])] = 0

In [202]:
charnum ={}
for k in character_list:
    charnum[k] = 0

In [203]:
for act_i in text_cleaned:
    for scene_j in act_i.split("Scene"):
        # recording character name and number of times they appear in a scene
        # dict format - Name: # of appeared times in scene_j 
        dict = {}
        for char in character_list:
            if scene_j.count(char) != 0:
                dict[char] = scene_j.count(char)
                charnum[char] += scene_j.count(char)
        for name1 in range(len(dict)-1):
            for name2 in range(name1+1,len(dict)):
                if dict[list(dict.keys())[name1]] <= dict[list(dict.keys())[name2]]:
                    connections[(list(dict.keys())[name1],list(dict.keys())[name2])] += dict[list(dict.keys())[name1]]
                else: 
                    connections[(list(dict.keys())[name1],list(dict.keys())[name2])] += dict[list(dict.keys())[name2]]

In [204]:
# remove names and connections with zero entries 
charnum = {name:count for name,count in charnum.items() if count != 0}
connections = {(source,target):weight for (source,target),weight in connections.items()
              if weight != 0}

In [205]:
# change the charnum and connections dictionaries into graph items 
# with correction factor for a clearer representation

graphitems = []

for k,v in charnum.items():
    dashnode = {'data': {'id': k,
                         'label': k.title(),
                         'size': str(v)}}
    graphitems.append(dashnode)

correction_factor = max(connections.values())

for k,v in connections.items():
    if v != 0:
        dashedge = {'data': {'source': k[0],
                             'target': k[1],
                             'weight': v/correction_factor}}
        graphitems.append(dashedge)

57


In [206]:
app = JupyterDash(__name__)

app.layout = html.Div([
    cyto.Cytoscape(
        layout={'name': 'cose'},
        elements=graphitems,
        style={'width': '100%', 'height': '750px'},
        stylesheet=[
            {
                'selector': 'node',
                'style': {
                    'content':'data(label)',
                    'text-halign':'center',
                    'text-valign':'center',
                    'width': 'data(size)',
                    'height': 'data(size)',
                    'font-size':5,
                    'color': 'black',
                    'background-color': 'red',
                    'text-outline-color': 'white',
                    'text-outline-width': 0.2,
                    'shape':'circle'
                }
            },
            {
                'selector':'edge',
                'style': {
                    'width':'data(weight)',
                    'line-color': 'blue',
                  }
            },
        ]
    )
])

app.run_server(debug=True)

Dash app running on https://jupyter.idre.ucla.edu/user/opheliadong@gmail.com/proxy/8050/


#### Comment on network density 
#### There are three dominating nodes, "Romeo," "Tybalt" and "Juliet." The rest of the nodes a significantly smaller. There isn't a smooth range in sizes, but more extreme ones, either very large or very big. 
#### The "important" nodes, as determined by size, would be "Romeo," "Tybalt" and "Juliet." 
#### There are two quite "important" realizations that we can derive from this visualization. The first is that the respective edge between "Romeo" and the other two dominating nodes are much stronger than the other. This means that the number of times "Romeo" appears in a scene where "Tybalt" also appears is very frequent. 
#### The second observation is that the "Juliet" node, though also large, is much smaller than "Romeo" and "Tybalt," which is strange since she is the main character that has her name stamped to the play. But in terms of her name's appearance in the play, it's even less than Tybalt. Perhaps Shakespeare wanted to put more emphasis on the male characters? I wonder that, if we do an analysis on the number of words Juliet spoke, is it significantly less than Romeo? Or Tybalt? If we analyze other plays from the same historical period, would we see similar trends, where female main characters being mentioned much less than her male equivalent parts? It is quite unexpected that a visualization between character names can reveal some aspects of gender inequality in theatre. 
